In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import random

import warnings
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.dpi'] = 100

## Load Dataset

In [2]:
df = pd.read_csv('pendigits.csv')
df.head()

,input1,input2,input3,input4,input5,input6,input7,input8,input9,input10,input11,input12,input13,input14,input15,input16,class
0,47,100,27,81,57,37,26,0,0,23,56,53,100,90,40,98,8
1,0,89,27,100,42,75,29,45,15,15,37,0,69,2,100,6,2
2,0,57,31,68,72,90,100,100,76,75,50,51,28,25,16,0,1
3,0,100,7,92,5,68,19,45,86,34,100,45,74,23,67,0,4
4,0,67,49,83,100,100,81,80,60,60,40,40,33,20,47,0,1


## Explore Data

In [6]:
def column_info(df):
   
    info_df = pd.DataFrame(columns=['Columns', 'Data_Type', 'Counts', 'Nulls', 'Uniques', 'Max', 'Min', 'Sample'])

    
    for col in df.columns:
        
        data_type = df[col].dtype
        
        count = df[col].count()
       
        nan_count = df[col].isnull().sum()
        
        unique_count = df[col].nunique()

        if pd.api.types.is_numeric_dtype(df[col]):
            
            max_value = df[col].max()
            min_value = df[col].min()
            non_nan_values = df[col].dropna().unique().tolist()
            random_value = random.choice(non_nan_values) if non_nan_values else None
            
            info_df = info_df._append({'Columns': col, 'Data_Type': data_type, 'Counts': count, 'Nulls': nan_count, 'Uniques': unique_count,
                                      'Max': max_value,
                                      'Min': min_value,
                                      'Sample': random_value}, ignore_index=True)
        else:
            
            sample_values = df[col].dropna().unique().tolist()
            
            info_df = info_df._append({'Columns': col, 'Data_Type': data_type, 'Counts': count, 'Nulls': nan_count, 'Uniques': unique_count,
                                      'Max': 'no value',
                                      'Min': 'no value',
                                      'Sample': random.choice(sample_values) if sample_values else None}, ignore_index=True)

    return info_df

In [7]:
column_info(df)

,Columns,Data_Type,Counts,Nulls,Uniques,Max,Min,Sample
0,input1,int64,10992,0,101,100,0,44
1,input2,int64,10992,0,96,100,0,33
2,input3,int64,10992,0,101,100,0,89
3,input4,int64,10992,0,98,100,0,74
4,input5,int64,10992,0,101,100,0,11
5,input6,int64,10992,0,101,100,0,59
6,input7,int64,10992,0,101,100,0,22
7,input8,int64,10992,0,101,100,0,97
8,input9,int64,10992,0,101,100,0,77
9,input10,int64,10992,0,101,100,0,4
